In [ ]:
import math
import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

import pandas as pd
import numpy as np
import flammkuchen as fl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ray

from spinorama.load import filter_graphs, norm_spl

from generate_common import custom_ray_init, cache_load

custom_ray_init({"--log-level": "INFO"})
df_all = cache_load({}, False)
ray.shutdown()

In [ ]:
speaker = "JBL 308P Mark ii"
# speaker = "Genelec 8341A"

df_klippel = df_all[speaker]["ASR"]["asr"]
spl_H = df_klippel["SPL Horizontal_unmelted"]
spl_V = df_klippel["SPL Vertical_unmelted"]


def a2v(angle):
    if angle == "Freq":
        return -1000
    elif angle == "On Axis":
        return 0
    iangle = int(angle[:-1])
    return iangle


def transform(spl, dbmax, clip_min, clip_max):
    if "-180°" not in spl.keys() and "180°" in spl.keys():
        spl["-180°"] = spl["180°"]
    df = spl.reindex(columns=sorted(spl.columns, key=lambda a: a2v(a))) - db_max
    # x,y,z
    freq = df.Freq
    angle = [a2v(i) for i in df.loc[:, df.columns != "Freq"].columns]
    selector = (df["Freq"] > 20) & (df["Freq"] < 20000)
    spl = df.loc[selector, df.columns != "Freq"].T.values
    # color
    color = np.clip(np.multiply(np.floor_divide(spl, 3), 3), clip_min, clip_max)
    return freq, angle, spl, color

In [ ]:
contour_start = -30
contour_end = 3

db_max = max(
    spl_H["On Axis"].max(),
    spl_V["On Axis"].max(),
)

x_h, y_h, z_h, c_h = transform(spl_H, db_max, contour_start, contour_end)
x_v, y_v, z_v, c_v = transform(spl_V, db_max, contour_start, contour_end)

print("{:0.1f} <= x_h <= {:0.1f}".format(np.min(x_h), np.max(x_h)))
print("{:0.1f} <= x_v <= {:0.1f}".format(np.min(x_v), np.max(x_v)))
print("{:0.1f} <= y_h <= {:0.1f}".format(np.min(y_h), np.max(y_h)))
print("{:0.1f} <= y_v <= {:0.1f}".format(np.min(y_v), np.max(y_v)))
print("{:0.1f} <= z_h <= {:0.1f}".format(np.min(z_h), np.max(z_h)))
print("{:0.1f} <= z_v <= {:0.1f}".format(np.min(z_v), np.max(z_v)))
print("{:0.1f} <= c_h <= {:0.1f}".format(np.min(c_h), np.max(c_h)))
print("{:0.1f} <= v_h <= {:0.1f}".format(np.min(c_v), np.max(c_v)))

colors = [
    [0, "rgb(0,0,168)"],
    [0.1, "rgb(0,0,200)"],
    [0.2, "rgb(0,74,255)"],
    [0.3, "rgb(0,152,255)"],
    [0.4, "rgb(74,255,161)"],
    [0.5, "rgb(161,255,74)"],
    [0.6, "rgb(255,255,0)"],
    [0.7, "rgb(234,159,0)"],
    [0.8, "rgb(255,74,0)"],
    [0.9, "rgb(222,74,0)"],
    [1, "rgb(253,14,13)"],
]

colorbar = dict(
    dtick=3,
    len=0.5,
    lenmode="fraction",
)

angle_list_3d = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]
angle_text_3d = [f"{a}°" for a in angle_list_3d]
freq_list_3d = [0, 5000, 10000, 15000]
freq_text_3d = ["{:d}k".format(a // 1000) for a in freq_list_3d]
spl_list_3d = [0, -5, -10, -15, -20, -25, -30, -35, -40, -45]
spl_text_3d = [f"{s}" if s > -45 else "" for s in spl_list_3d]

for freq, angle, spl, color in ((x_h, y_h, z_h, c_h), (x_v, y_v, z_v, c_v)):
    fig = go.Figure()
    trace = go.Surface(
        x=freq,
        y=angle,
        z=spl,
        showscale=True,
        autocolorscale=False,
        colorscale=colors,
        surfacecolor=color,
        colorbar=colorbar,
        cmin=contour_start,
        cmax=contour_end,
        hovertemplate="Freq: %{x:.0f}Hz<br>Angle:  %{y}°<br> SPL: %{z:.1f}dB<br>",
    )

    fig.add_trace(trace)

    fig.update_layout(
        title="Speakers {0}".format(speaker),
        autosize=False,
        width=600,
        height=700,
        scene=dict(
            xaxis=dict(
                title="Freq. (Hz)",
            ),
            yaxis=dict(
                range=[-180, 180],
                showline=True,
                tickvals=angle_list_3d,
                ticktext=angle_text_3d,
                title="Angle",
            ),
            zaxis=dict(
                range=[-45, 5],
                title="SPL",
                showline=True,
                tickvals=spl_list_3d,
                ticktext=spl_text_3d,
            ),
            # camera = dict(
            #    up=dict(x=0, y=0, z=0),
            #    center=dict(x=0, y=0, z=0),
            #    eye=dict(x=1.25, y=1.25, z=1.25)
            # ),
        ),
    )
    fig.update_traces(contours_z=dict(show=True, project_z=True))
    fig.show()